# Introduction to Python part XII (And a discussion of stochastic differential equations)

## Activity 1: Discussion stochastic differential equations
  * What is Lipshitz continuity?  How can this be interpreted relative to differentiability? How is this related to initial value problems?
  * What do we refer to when we say a Dirac "measure" or Dirac "distribution"?  How is this related to an initial value problem?
  * What is the formal equation of an SDE?  What is the "drift" and what is the "diffusion"?
  * What is "additive noise"?  What is special about such a system?
  * What is a path solution to an SDE?  How is this related to the Fokker-Plank equations?

## Activity 2: Defensive Programming


Our previous lessons have introduced the basic tools of programming: variables and lists, file I/O, loops, conditionals, and functions. What they haven’t done is show us how to tell whether a program is getting the right answer, and how to tell if it’s still getting the right answer as we make changes to it.

To achieve that, we need to:

* Write programs that check their own operation.
* Write and run tests for widely-used functions.
* Make sure we know what “correct” actually means.

The good news is, doing these things will speed up our programming, not slow it down. As in real carpentry — the kind done with lumber — the time saved by measuring carefully before cutting a piece of wood is much greater than the time that measuring takes.



The first step toward getting the right answers from our programs is to assume that mistakes will happen and to guard against them. This is called defensive programming, and the most common way to do it is to add assertions to our code so that it checks itself as it runs. An assertion is simply a statement that something must be true at a certain point in a program. When Python sees one, it evaluates the assertion’s condition. If it’s true, Python does nothing, but if it’s false, Python halts the program immediately and prints the error message if one is provided. For example, this piece of code halts as soon as the loop encounters a value that isn’t positive:



In [2]:
numbers = [1.5, 2.3, 0.7, -0.001, 4.4]
total = 0.0
for num in numbers:
    assert num > 0.0, 'Data should only contain positive values'
    total += num
print('total is:', total)

AssertionError: Data should only contain positive values

Programs like the Firefox browser are full of assertions: 10-20% of the code they contain are there to check that the other 80–90% are working correctly. Broadly speaking, assertions fall into three categories:

* A precondition is something that must be true at the start of a function in order for it to work correctly.
* A postcondition is something that the function guarantees is true when it finishes.
* An invariant is something that is always true at a particular point inside a piece of code.

For example, suppose we are representing rectangles using a tuple of four coordinates (x0, y0, x1, y1), representing the lower left and upper right corners of the rectangle. In order to do some calculations, we need to normalize the rectangle so that the lower left corner is at the origin and the longest side is 1.0 units long. This function does that, but checks that its input is correctly formatted and that its result makes sense:

In [3]:
def normalize_rectangle(rect):
    """Normalizes a rectangle so that it is at the origin and 1.0 units long on its longest axis.
    Input should be of the format (x0, y0, x1, y1).
    (x0, y0) and (x1, y1) define the lower left and upper right corners
    of the rectangle, respectively."""
    assert len(rect) == 4, 'Rectangles must contain 4 coordinates'
    x0, y0, x1, y1 = rect
    assert x0 < x1, 'Invalid X coordinates'
    assert y0 < y1, 'Invalid Y coordinates'

    dx = x1 - x0
    dy = y1 - y0
    if dx > dy:
        scaled = float(dx) / dy
        upper_x, upper_y = 1.0, scaled
    else:
        scaled = float(dx) / dy
        upper_x, upper_y = scaled, 1.0

    assert 0 < upper_x <= 1.0, 'Calculated upper X coordinate invalid'
    assert 0 < upper_y <= 1.0, 'Calculated upper Y coordinate invalid'

    return (0, 0, upper_x, upper_y)

The preconditions on lines 6, 8, and 9 catch invalid inputs:

In [4]:
print(normalize_rectangle( (0.0, 1.0, 2.0) )) # missing the fourth coordinate

AssertionError: Rectangles must contain 4 coordinates

In [5]:
print(normalize_rectangle( (4.0, 2.0, 1.0, 5.0) )) # X axis inverted


AssertionError: Invalid X coordinates

In [6]:
print(normalize_rectangle( (0.0, 0.0, 1.0, 5.0) ))


(0, 0, 0.2, 1.0)


but if we normalize one that’s wider than it is tall, the assertion is triggered:


In [7]:
print(normalize_rectangle( (0.0, 0.0, 5.0, 1.0) ))


AssertionError: Calculated upper Y coordinate invalid

Re-reading our function, we realize that line 14 should divide `dy` by `dx` rather than `dx` by `dy`. In a Jupyter notebook, you can display line numbers by typing `Ctrl+M` followed by `L`. If we had left out the assertion at the end of the function, we would have created and returned something that had the right shape as a valid answer, but wasn’t. Detecting and debugging that would almost certainly have taken more time in the long run than writing the assertion.

But assertions aren’t just about catching errors: they also help people understand programs. Each assertion gives the person reading the program a chance to check (consciously or otherwise) that their understanding matches what the code is doing.

Most good programmers follow two rules when adding assertions to their code. The first is, fail early, fail often. The greater the distance between when and where an error occurs and when it’s noticed, the harder the error will be to debug, so good code catches mistakes as early as possible.

The second rule is, turn bugs into assertions or tests. Whenever you fix a bug, write an assertion that catches the mistake should you make it again. If you made a mistake in a piece of code, the odds are good that you have made other mistakes nearby, or will make the same mistake (or a related one) the next time you change it. Writing assertions to check that you haven’t regressed (i.e., haven’t re-introduced an old problem) can save a lot of time in the long run, and helps to warn people who are reading the code (including your future self) that this bit is tricky.